Acknowledge that the dataset herein is miniature and can be toyed with on Jupyter nb using a local machine, perhaps with single GPU. More realistic datasets would require API interfacing with a cluster and data engineering pipelines to display distributions of billions of params over time, or to run robust imputation methods for missing data as standalone endeavors before one even considers modeling.

In [69]:
import pandas as pd

In [70]:
trainRaw = pd.read_csv('../data/train_sparked.csv')
testRaw = pd.read_csv('../data/test_sparked.csv')

In [71]:
trainRaw

,SellerCity,SellerIsPriv,SellerListSrc,SellerRating,SellerRevCnt,SellerState,SellerZip,VehCertified,VehColorExt,VehColorInt,...,VehModel,VehPriceLabel,VehTransmission,VehYear,Vehicle_Trim,Dealer_Listing_Price,NumOwners,Token_VehSellerNotes,Token_VehFeats,Token_SellerName
0,warren,False,inventorycommandcenter,5.0,32,MI,48091,False,['White'],black,...,grandcherokee,fairprice,Automatic 8-Speed,2015,High Altitude,30990.0,1.0,"[0.298429012298584,-0.17518772184848785,0.0321...","[-0.08651182216338137,0.15116102536293594,-0.1...","[-0.03862341493368149,-0.033364564180374146,0...."
1,waukesha,False,jeepcertifiedprogram,4.8,1405,WI,53186,True,"['Brilliant', 'Black', 'Crystal', 'Pearlcoat']",black,...,grandcherokee,gooddeal,8-Speed Automatic w/OD,2015,Laredo,23249.0,1.0,"[0.18062071350344397,-0.2928671848515535,0.071...","[0.03729163779838321,-0.02813182579120621,-0.0...","[0.006213635206222534,0.010602051392197609,0.0..."
2,wentzville,False,inventorycommandcenter,4.4,21,MO,63385,False,"['Diamond', 'Black', 'Crystal', 'Pearlcoat']",black,...,grandcherokee,gooddeal,Automatic,2018,Limited,31977.0,1.0,"[0.2255005446366138,-0.2244718712237146,0.0563...","[-0.05632210763938287,0.21207473596886678,-0.2...","[-0.04601607844233513,0.004166889004409313,0.0..."
3,fayetteville,False,homenetautomotive,3.7,74,AR,72703,False,"['Radiant', 'Silver', 'Metallic']",cirrus,...,xt5,gooddeal,8-Speed Automatic,2018,Luxury,33495.0,1.0,"[0.1279906820850212,-0.22962800957587304,0.068...","[0.07798656957367292,-0.10927963254149431,-0.1...","[-0.012232745066285133,-0.009240418672561646,0..."
4,olean,False,digitalmotorworksdmi,3.0,51,NY,14760,False,"['Billet', 'Silver', 'Metallic', 'Clearcoat']",black,...,grandcherokee,gooddeal,8-Speed Automatic,2018,Limited,27900.0,1.0,"[0.17225867569134806,-0.15315184596989115,0.09...","[0.04796752968936094,-0.20832853338548113,0.00...","[-0.02215905860066414,-0.014781760983169079,0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6016,columbus,False,digitalmotorworksdmi,1.5,6,OH,43235,False,['Black'],black,...,grandcherokee,gooddeal,8-Speed Automatic,2015,Limited,25998.0,1.0,"[0.24771042000925678,-0.3370496009963311,-0.14...","[-0.005968482575068871,-0.21977887004613875,0....","[-0.007896220311522484,-0.02153726853430271,0...."
6017,indianapolis,False,digitalmotorworksdmi,3.3,16,IN,46280,False,['Gray'],black,...,grandcherokee,fairprice,8-Speed Automatic,2015,Limited,31998.0,1.0,"[0.2718227221917959,-0.3324910427018264,-0.157...","[-0.005968482575068871,-0.21977887004613875,0....","[0.00257093901745975,-0.0344950333237648,-0.02..."
6018,dublin,False,digitalmotorworksdmi,4.1,20,OH,43017,True,['Black'],jetblack,...,xt5,gooddeal,8-Speed Automatic,2018,Luxury,35674.0,0.0,"[0.26451578798393405,-0.12848351902707858,0.08...","[0.005631768493913114,-0.16371686203638092,-0....","[0.028927350416779518,-0.0027275383472442627,0..."
6019,sandusky,False,digitalmotorworksdmi,4.9,278,OH,44870,False,['Black'],jetblack,...,xt5,greatdeal,8-Speed Automatic,2017,Luxury,31995.0,2.0,"[0.15525871395241939,-0.11167732475877024,0.08...","[-0.052938118926249444,-0.1297987708239816,-0....","[0.024324465543031693,0.047628313302993774,-0...."


In [72]:
testRaw

,SellerCity,SellerIsPriv,SellerListSrc,SellerRating,SellerRevCnt,SellerState,SellerZip,VehCertified,VehColorExt,VehColorInt,...,VehMake,VehMileage,VehModel,VehPriceLabel,VehTransmission,VehYear,NumOwners,Token_VehSellerNotes,Token_VehFeats,Token_SellerName
0,seneca,False,homenetautomotive,2.5,59,SC,29678,False,"['Stellar', 'Black', 'Metallic']",cirrus,...,Cadillac,13625,xt5,gooddeal,8-Speed Automatic,2018,1.0,"[0.06540291840568759,-0.06187705641898972,-0.0...","[0.0367064405646589,0.010015353974368837,-0.11...","[-0.001974827144294977,0.011606967076659203,0...."
1,bedford,False,inventorycommandcenter,4.7,2116,OH,44146,False,"['True', 'Blue', 'Pearlcoat']",black,...,Jeep,42553,grandcherokee,gooddeal,8-Speed Automatic,2017,1.0,"[0.06442540839225763,-0.04858947691862427,-0.0...","[-0.087644991219828,0.0029494149708434152,-0.1...","[-0.049153637140989304,-0.03347434476017952,0...."
2,webster,False,jeepcertifiedprogram,3.9,46,NY,14580,True,"['Brilliant', 'Black', 'Crystal', 'Pearlcoat']",black,...,Jeep,48951,grandcherokee,gooddeal,8-Speed Automatic,2015,1.0,"[0.04382465131849705,-0.12188866790033345,-0.0...","[-0.05273406638298184,-0.1287083673523739,-0.0...","[0.03288218379020691,-0.0013011932605877519,-0..."
3,louisville,False,digitalmotorworksdmi,4.5,1075,KY,40222,False,"['Brilliant', 'Black', 'Crystal', 'Pearlcoat']",black,...,Jeep,44179,grandcherokee,gooddeal,8-Speed Automatic,2015,1.0,"[0.0009591081739449193,-0.008942851241118016,-...","[-0.12472875254849593,-0.14115160865088303,-0....","[-0.03471272066235542,0.00036557912244461477,-..."
4,palmyra,False,digitalmotorworksdmi,4.6,162,NJ,8065,False,"['Harbor', 'Blue', 'Metallic']",jetblack,...,Cadillac,22269,xt5,gooddeal,8-Speed Automatic,2018,1.0,"[0.0033283630052603173,-0.05054311789590358,-0...","[0.030946276972399038,-0.08527886542929884,-0....","[-0.004474562592804432,-0.006331950426101685,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950,forestpark,False,homenetautomotive,4.8,1081,IL,60130,False,"['Granite', 'Crystal', 'Metallic', 'Clearcoat']",black,...,Jeep,24744,grandcherokee,gooddeal,8-Speed Automatic w/OD,2017,1.0,"[-0.04286873616349169,0.0076026679144549966,-0...","[-0.00015769165981075037,-0.016996438088624374...","[-0.015910614281892776,-0.020877087488770485,0..."
951,tampa,False,inventorycommandcenter,4.0,240,FL,33614,False,['Black'],jetblack,...,Cadillac,5699,xt5,gooddeal,8-Speed Automatic,2017,1.0,"[0.025183369860869942,-0.05851111839420825,-0....","[-0.09635099242715274,0.06970605539048419,0.06...","[-0.005991202779114246,0.016028672456741333,0...."
952,tampa,False,homenetautomotive,5.0,134,FL,33612,False,"['Radiant', 'Silver', 'Metallic']",jetblack,...,Cadillac,17985,xt5,gooddeal,8-Speed Automatic,2018,1.0,"[-0.00712872901902446,-0.011860699366212854,-0...","[0.019578716019168496,0.00963064911775291,-0.1...","[-0.014352565631270409,0.04984908178448677,0.0..."
953,hamburg,False,inventorycommandcenter,3.8,7,NY,14075,False,"['Diamond', 'Black']",black,...,Jeep,27,grandcherokee,fairprice,8-Speed Automatic,2018,1.0,"[0.05813311710953713,-0.08677019542083145,-0.1...","[-0.13011326231062412,0.08693666085600854,0.03...","[0.0388680174946785,-0.03917462378740311,0.034..."


In [73]:
len(trainRaw['Token_VehFeats'].unique()), len(trainRaw['Token_SellerName'].unique())

(811, 2315)

In [74]:
trainRaw.info(verbose=False), testRaw.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6021 entries, 0 to 6020
Columns: 27 entries, SellerCity to Token_SellerName
dtypes: bool(2), float64(4), int64(4), object(17)
memory usage: 1.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955 entries, 0 to 954
Columns: 25 entries, SellerCity to Token_SellerName
dtypes: bool(2), float64(3), int64(4), object(16)
memory usage: 173.6+ KB


(None, None)

In [75]:
null_counts = trainRaw.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

VehColorInt             530
VehPriceLabel           269
VehDriveTrain           211
VehHistory              185
NumOwners               185
VehEngine               169
Vehicle_Trim            153
VehColorExt              62
Dealer_Listing_Price     50
VehTransmission          30
dtype: int64

In [76]:
null_counts = testRaw.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

VehColorInt        81
VehDriveTrain      36
VehPriceLabel      33
VehEngine          30
VehHistory         25
NumOwners          25
VehTransmission     6
dtype: int64